In [8]:
!pip install ultralytics

In [16]:
import os
import yaml
from ultralytics import YOLO
import glob
import torch

In [11]:
def create_labels_for_no_defect_images(images_dir, labels_dir, class_id=4):
    """
    Создает пустые файлы меток для изображений без дефектов
    """
    # Получаем список всех изображений
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    image_files = []

    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(images_dir, ext)))
        image_files.extend(glob.glob(os.path.join(images_dir, ext.upper())))

    # Создаем метки для каждого изображения
    for image_path in image_files:
        # Получаем имя файла без расширения
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        label_path = os.path.join(labels_dir, f"{image_name}.txt")

        # Если файл метки не существует, создаем пустой
        if not os.path.exists(label_path):
            with open(label_path, 'w') as f:
                # Пустой файл - нет bounding boxes
                pass
            print(f"Создан пустой файл метки: {label_path}")

In [12]:
def setup_dataset_yaml(data_dir, output_path='/content/drive/MyDrive/AI/apples_yolo_dataset/dataset.yaml'):
    """
    Создает YAML файл конфигурации датасета
    """
    config = {
        'path': data_dir,
        'train': 'images/train',
        'val': 'images/val',
        'names': {
            0: 'critical_defect',
            1: 'significant_defect',
            2: 'minor_defect',
            3: 'no_defect'
        }
    }

    with open(output_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)

    print(f"Создан файл конфигурации: {output_path}")
    return output_path

In [17]:
def train_yolov8():
    # Пути к данным
    data_dir = '/content/drive/MyDrive/AI/apples_yolo_dataset'
    train_images_dir = os.path.join(data_dir, 'images/train')
    val_images_dir = os.path.join(data_dir, 'images/val')
    train_labels_dir = os.path.join(data_dir, 'labels/train')
    val_labels_dir = os.path.join(data_dir, 'labels/val')

    # Создаем метки для изображений без дефектов
    print("Создание меток для train изображений без дефектов...")
    create_labels_for_no_defect_images(train_images_dir, train_labels_dir)

    print("Создание меток для val изображений без дефектов...")
    create_labels_for_no_defect_images(val_images_dir, val_labels_dir)

    # Создаем конфигурационный файл датасета
    yaml_path = setup_dataset_yaml(data_dir)

    # Загружаем предобученную модель YOLOv8
    print("Загрузка модели YOLOv8...")
    model = YOLO('yolov8n.pt')  # можно использовать 'yolov8s.pt', 'yolov8m.pt' и т.д.

    # Настройки обучения
    training_config = {
        'data': yaml_path,
        'epochs': 100,
        'imgsz': 640,
        'batch': 16,
        'workers': 4,
        'patience': 0,
        'save': True,
        'exist_ok': True,
        'pretrained': True,
        'optimizer': 'auto',
        'lr0': 0.01,
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3.0,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 7.5,
        'cls': 0.5,
        'dfl': 1.5,
    }

    # Запускаем обучение
    print("Начало обучения...")
    results = model.train(**training_config)

    print("Обучение завершено!")
    return model, results